![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft-small-logo.png)
## Run SQL Server 2019 CTP 3.1 container image with Docker
This notebook will use Docker to pull and run the SQL Server 2019 CTP 3.1 container image and connect to it in Azure Data Studio

### Dependencies
- Docker Engine. For more information, see [Install Docker](https://docs.docker.com/engine/installation/).

### Check dependencies

In [1]:
def run_command():
    print("Executing: " + cmd)
    !{cmd}
    if _exit_code != 0:
        raise SystemExit(f'Shell command:\n\n\t{cmd}\n\nreturned non-zero exit code: ' + str(_exit_code) + '.\n')
    print(f'Successfully executed: {cmd}')

cmd = 'docker version'
run_command()

### List existing containers
You can view the ports that have been used by existing containers

In [3]:
md = f'docker ps -a'
run_command()

### Required information

In [4]:
import getpass
password_name = 'SQL Server sa account password'
sql_password = getpass.getpass(prompt = password_name)
password_confirm = getpass.getpass(prompt = f'Confirm {password_name}')
if sql_password != password_confirm:
    raise SystemExit(f'{password_name} does not match the confirmation password')
print(f'{password_name}: ******')
sql_port = input('SQL Server port, default value is 1433')
if len(sql_port) == 0:
    sql_port = '1433'
print(f'Port: {sql_port}')

### Pull the container image

In [5]:
cmd = f'docker pull mcr.microsoft.com/mssql/server:2019-latest'
run_command()

### Start a new container

In [6]:
import time
container_name = 'sql2019-' + time.strftime("%Y%m%d%H%M%S", time.localtime())
print('New container name: ' + container_name)
cmd = f'docker run -e ACCEPT_EULA=Y -e "SA_PASSWORD={sql_password}" -p {sql_port}:1433 --name {container_name} -d mcr.microsoft.com/mssql/server:2019-latest'
run_command()

### List all the containers

In [7]:
cmd = f'docker ps -a'
run_command()

### Connect to SQL Server in Azure Data Studio
It might take a couple minutes for SQL Server to launch

In [8]:
from IPython.display import *
display(HTML("<a href=\"command:azdata.connect?{&quot;serverName&quot;:&quot;localhost,"+sql_port+"&quot;,&quot;providerName&quot;:&quot;MSSQL&quot;, &quot;authenticationType&quot;:&quot;SqlLogin&quot;,&quot;userName&quot;:&quot;sa&quot;,&quot;password&quot;:&quot;"+sql_password+"&quot;}\">Click here to connect to SQL Server</a>"))

### Stop and remove the container

In [9]:
stop_container_command = f'docker stop {container_name}'
remove_container_command = f'docker rm {container_name}'
display(HTML("Use this link to: <a href=\"command:workbench.action.terminal.focus\">open the terminal window in Azure Data Studio</a> and use the links below to paste the command to the terminal."))
display(HTML("Stop the container: <a href=\"command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22"+stop_container_command.replace(" ","%20")+"%22%7D\">" + stop_container_command + "</a>"))
display(HTML("Remove the container: <a href=\"command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22"+remove_container_command.replace(" ","%20")+"%22%7D\">" + remove_container_command + "</a>"))